# Bayesian statistics

Bayesian statistics allows priors (null hypotheses) and posteriors (effect sizes) to take on *distributions*. This allows us to assign degrees of certainty to events and statements. 

Typically, a harsh contrast is drawn between frequentists (null hypothesis, p-values) and Bayesians, and this is probably because they address different *questions*.

The frequentist asks: 
* do I have sufficient data to reject the null hypothesis -- that there is no difference between my observed data and a straw-man (null) hypothesis?

The Bayesian asks:
* what can I learn about my data and the underlying system that was used to generate it?

The divide arises from different ways of seein the natural world. A frequentist will assume that *there is a true value of a system* and the data we use to estimate the system are noisy. A Bayesian would say that we would do better to think of the data as *fixed*, and assume that the *system is noisy*. 

This is best illustrated with an example.

In [ ]:
# This function is a t-test-like operation using Bayesian statistics
def bayes_ttest(groups=None, N=40, show=False):
  """
  Run a Bayesian t-test on sample or true data.
  """
  if groups is None: # Generate some data
    group1, group2 = gen_data(N=40)
  elif len(groups) != 2:
    print('T-test requires only 2 groups, not %i' %len(groups))
    return None
  else:
    group1, group2 = groups
  
  pooled = np.concatenate((group1, group2)) # Pooled data
  # Establish priors
  mu1 = pm.Normal("mu_1", mu=pooled.mean(), tau=1.0/pooled.var()/N)
  mu2 = pm.Normal("mu_2", mu=pooled.mean(), tau=1.0/pooled.var()/N)
  sig1 = pm.Uniform("sigma_1",lower=pooled.var()/1000.0,upper=pooled.var()*1000)  
  sig2 = pm.Uniform("sigma_2",lower=pooled.var()/1000.0,upper=pooled.var()*1000)
  v = pm.Exponential("nu", beta=1.0/29)
  
  # Set up posterior distribution
  t1 = pm.NoncentralT("t_1", mu=mu1, lam=1.0/sig1, nu=v, value=group1,
                      observed=True)
  t2 = pm.NoncentralT("t_1", mu=mu2, lam=1.0/sig2, nu=v, value=group2,
                      observed=True)
  
  # Generate the model
  model = pm.Model( [t1, mu1, sig1, t2, mu2, sig2, v] ) # Push priors
  mcmc = pm.MCMC(model) # Generate MCMC object
  mcmc.sample(40000, 10000, 2) # Run MCMC sampler # "trace"
  
  # Get the numerical results
  mus1 = mcmc.trace('mu_1')[:]  
  mus2 = mcmc.trace('mu_2')[:]  
  sigmas1 = mcmc.trace('sigma_1')[:]  
  sigmas2 = mcmc.trace('sigma_2')[:]  
  nus = mcmc.trace('nu')[:] 
  diff_mus = mus1-mus2  # Difference in mus
  diff_sigmas = sigmas1-sigmas2  
  normality = np.log(nus)  
  effect_size = (mus1-mus2)/np.sqrt((sigmas1**2+sigmas2**2)/2.)  
  print('\n   Group 1 mu: %.4f\n   Group 2 mu: %.4f\n   Effect size: %.4f'
        %(mus1.mean(), mus2.mean(), effect_size.mean()))
  
  if show: # Plot some basic metrics if desired
    from pymc.Matplot import plot as mcplot
    # mcplot(mcmc) # This plots 5 graphs, only useful as a benchmark.
    
    # Finally, what can this tell is about the null hypothesis?
    # Split distribution
    fig2 = plt.figure() 
    ax2 = fig2.add_subplot(121)
    minx = min(min(mus1),min(mus2))  
    maxx = max(max(mus1),max(mus2))  
    xs = np.linspace(minx,maxx,1000)
    gkde1 = stats.gaussian_kde(mus1)  
    gkde2 = stats.gaussian_kde(mus2)
    ax2.plot(xs,gkde1(xs),label='$\mu_1$')  
    ax2.plot(xs,gkde2(xs),label='$\mu_2$')  
    ax2.set_title('$\mu_1$ vs $\mu_2$')
    ax2.legend()
    
    # Difference of mus
    ax3 = fig2.add_subplot(122)
    minx = min(diff_mus)  
    maxx = max(diff_mus)  
    xs = np.linspace(minx,maxx,1000)  
    gkde = stats.gaussian_kde(diff_mus)
    ax3.plot(xs,gkde(xs),label='$\mu_1-\mu_2$')  
    ax3.legend()
    ax3.axvline(0, color='#000000',alpha=0.3,linestyle='--')
    ax3.set_title('$\mu_1-\mu_2$')
    plt.show()
  
  return


In [ ]:
# Actually run it with our test groups, A and B
bayes_ttest(groups=[A,B], show=True)

Here we can see that there is a difference in the means of A and B, but unlike with a simple t-test we get an estimate of the confidence of this effect size.

## Behind the scenes

Bayesian statistics are most commonly run using Markov Chain Monte Carlo simulation methods. 

MCMC takes a data set and treats it as the operation to act on a prior distribution. MCMC is interested in re-creating the parameters used to generate the dataset. For a normal distribution (as above), this means a $\mu$ (sample mean) and $\sigma$ (standard deviation/variance). 

MCMC (hopefully) converges on parameters by repeatedly applying a mathematically complex but heuristic algorithm. In order to arrive at the posterior distribution, mu and sigma will take on values that approximate that of the data.

Prior distributions of $\mu$ and $\sigma$ are fed into the MCMC. If we have no reason to suspect a bias, these can be uninformative priors, or uniform, spanning some range that will include the mean of the data.


In [ ]:
plt.plot(range(80), [1./80. for i in range(80)], lw=2.)
plt.fill_between(range(80), [1./80. for i in range(80)], np.zeros(80), 
                 facecolor='blue', alpha=0.4)
plt.ylim([0,0.25])

Whereas for $\sigma$ it makes less sense to choose a uniform prior (though we certainly could). We might instead be inclined to believe $\sigma$ is closer to 0 than to 80, and so we make its prior an exponential distribution.

In [ ]:
x = np.linspace(0., 30., 300)
E = [np.exp(-(i/5.))/5. for i in x]
plt.plot(x, E, linewidth=2.)
plt.fill_between(x, E, np.zeros(len(E)), facecolor='b', alpha=0.4)
plt.ylim([0,0.25])
plt.xlim([0,80])

These distributions are approximated with randomly chosen numbers from the generator functions (beta, binomial, Bernoulli, Poisson, exponential) in a method known as Monte Carlo sampling.

Here is a sample random exponential distribution that 

In [ ]:
E_data = np.random.exponential(5, 200)
plt.hist(E_data, bins=20, facecolor='b', edgecolor='none', 
             alpha=0.4, normed=1)
plt.ylim([0,0.25])
plt.xlim([0,80])

The job of the MCMC sampler is to use the given data to transform the prior, all the while tracking how the parameters ($\mu$ and $\sigma$ here) respond to shifts in the posterior.

In general, MCMC flows:

1. Prior distribution is subjected to data operations:
 * a random data point is chosen and can move one unit direction
 * the random data point moves probabilistically based on which direction has more observed data points
 * the random data point can move or stay the same, and has no memory of where it's been (Markovian)
2. This is repeated several thousand times, and each time creates a distribution

At the end, we have a posterior distribution. This distribution is much  more informative than frequentist statistical tests. We can ask more detailed questions about the data, such as

* what is the probability of the data having a mean of $\mu_x$?
* what is the likelihood of the data being greater than $x_0%?
* what are the credible regions?
* what are the credible regions of the effect sizes?

Most valuably, we have probabilities for all of these questions and numbers, not just binary yes/no.

## P-substitute

We also have a p-value substitute ready at hand. A simple alternative that is absolutely viable is the percent of samples that fall beyond a benchmark. 

In [ ]:
# Simple p-substitute values
locs = [1.5, 5.5, 11.5]
scales = [1., 3., 5.]
fig = plt.figure()
for s in range(3):
    ax = fig.add_subplot(1,3,s+1)
    x = np.random.normal(loc=locs[s], scale=scales[s], size=400)
    cnts, bins, patches = ax.hist(x, bins=20, facecolor='gray', 
                                  edgecolor='white', alpha=0.4)
    bincents = 0.5 * (bins[:-1] + bins[1:]) # Bin centers
    cols, cnts_sofar = [], 0. # Colors
    for b in bincents:
        if b < 0.:
            cols.append('red')
        else:
            cols.append('blue')
            cnts_sofar += cnts[list(bincents).index(b)]
    # Set the facecolors
    for c, p in zip(cols, patches): 
        plt.setp(p, 'facecolor', c)
    # Mark the benchmark
    ax.plot([0,0], [0,max(cnts)], '--', color='r', linewidth=2.)
    ax.set_title('%.2f %% above 0' %float(cnts_sofar/sum(cnts)))


## Sources

** Chan, C** *Data analysis with Python -- Practical computing for biologists.* http://people.duke.edu/~ccc14/pcfb/analysis.html

**Manly, B** *Randomization, Bootstrap and Monte Carlo methods in Biology.*

**Pilon, CD** *Probabilistic programming and Bayesian methods for Hackers.* http://nbviewer.jupyter.org/github/CamDavidsonPilon/Probabilistic-Programming-and-Bayesian-Methods-for-Hackers/tree/master/

**Vanderplas, J** *Frequentism and Bayesianism: A Python-driven Primer.* http://jakevdp.github.io/blog/2014/06/14/frequentism-and-bayesianism-4-bayesian-in-python/
(Also an associated paper on arXiv: http://arxiv.org/abs/1411.5018)